In [54]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
!ls

BOW -- Logistic Regression.ipynb  building-a-prediction-model.ipynb
README.md                         reviewsV1.db
bow.ipynb


In [3]:
with sqlite3.connect('reviewsV1.db') as conn:
    data = pd.read_sql_query('SELECT * FROM Review', conn)

In [4]:
data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Polarity
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1303862400,Good Quality Dog Food,good stew smells bought looks vitality appreci...,positive
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1346976000,Not as Advertised,small product peanuts arrived labeled jumbo si...,negative
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1219017600,"""Delight"" says it all",treat liberally citrus brother nuts highly cas...,positive
3,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,1307923200,Cough Medicine,secret ingredient flavor root cherry looking o...,negative
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1350777600,Great taffy,taffy wide delivery assortment great yummy,positive


In [5]:
data.sort_values(by='Time', inplace=True)
data.reset_index(drop=True, inplace=True)
TRAIN_SIZE = int(data.shape[0] * 0.7)
TEST_SIZE = data.shape[0] - TRAIN_SIZE

In [6]:
TRAIN_SIZE

254883

In [7]:
TEST_SIZE

109236

In [8]:
data_train = data[0: TRAIN_SIZE]
data_test = data[TRAIN_SIZE:]

In [9]:
assert(data_train.shape[0] == TRAIN_SIZE)
assert(data_test.shape[0] == TEST_SIZE)
assert(data.Time.max() == data_test.Time.reset_index(drop=True)[TEST_SIZE -1])
assert(data.Time.min() == data_train.Time.reset_index(drop=True)[0])


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cunt = CountVectorizer(binary=True, max_features=5000)  # For Bernouli's Naive Bayes
cunt.fit(data_train.Text)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
Dtrain = cunt.transform(data_train.Text)
Dtrain.get_shape()

(254883, 5000)

In [13]:
Dtest = cunt.transform(data_test.Text)
Dtest.get_shape()

(109236, 5000)

In [1]:
from sklearn.datasets import load_breast_cancer

In [2]:
data = load_breast_cancer()

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [160]:
params = {
    'C': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
}

In [161]:
from sklearn.model_selection import train_test_split

In [162]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data.data, data.target, test_size=0.2)

In [163]:
estimator = LogisticRegression(random_state=42)

In [164]:
print(estimator)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [165]:
grid = GridSearchCV(estimator=estimator,
                    param_grid=params,
                    scoring={'accuracy', 'precision', 'f1', 'recall'},
                    refit='f1', # Because we are using multiple evaluation metrics
                    cv=10,
                    return_train_score=True,
                    verbose=2)

In [166]:
grid.fit(Xtrain, ytrain)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV] C=0.0001 ........................................................
[CV] ......................................... C=0.0001, total=   0.0s
[CV] C=0.0001 ........................................................
[CV] ......................................... C=0.0001, total=   0.0s
[CV] C=0.0001 ........................................................
[CV] ......................................... C=0.0001, total=   0.0s
[CV] C=0.0001 ........................................................
[CV] ......................................... C=0.0001, total=   0.0s
[CV] C=0.0001 ........................................................
[CV] ......................................... C=0.0001, total=   0.0s
[CV] C=0.0001 ........................................................
[CV] ......................................... C=0.0001, total=   0.0s
[CV] C=0.0001 ........................................................
[CV] ...........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] .

[CV] ........................................ C=10000.0, total=   0.0s
[CV] C=10000.0 .......................................................
[CV] ........................................ C=10000.0, total=   0.0s
[CV] C=10000.0 .......................................................
[CV] ........................................ C=10000.0, total=   0.0s
[CV] C=10000.0 .......................................................
[CV] ........................................ C=10000.0, total=   0.0s
[CV] C=10000.0 .......................................................
[CV] ........................................ C=10000.0, total=   0.0s
[CV] C=10000.0 .......................................................
[CV] ........................................ C=10000.0, total=   0.0s
[CV] C=10000.0 .......................................................
[CV] ........................................ C=10000.0, total=   0.0s


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    0.9s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit='f1', return_train_score=True,
       scoring={'accuracy', 'precision', 'recall', 'f1'}, verbose=2)

In [169]:
def get_summary(grid):
    parms = list(grid.param_grid.keys())
    columns = ['Iter #']
    scorings = grid.scoring
    for scoring in scorings:
        for typ in ['train', 'test']:
            columns.append(f'{typ}_{scoring}')
    columns.extend(['fit_time', 'score_time'])
    for param in grid.param_grid.keys():
        columns.append(param)
    df = pd.DataFrame(columns=columns)
    cv_res = grid.cv_results_
    for col in columns[1:-len(parms)]:
        df[col] = cv_res['mean_' + col]
    items = 1
    for k, v in grid.param_grid.items():
        items *= len(v)
    df['Iter #'] = np.array(range(items)) + 1
    res = defaultdict(list)
    for each in grid.cv_results_['params']:
        for p in parms:
            res[p].append(each[p])
    for k, v in res.items():
        df[k] = v
    return df

In [170]:
get_summary(grid)

,Iter #,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f1,test_f1,fit_time,score_time,C
0,1,0.928448,0.925275,0.909251,0.908521,0.981283,0.978414,0.943887,0.941725,0.003298,0.002186,0.0001
1,2,0.933580,0.934066,0.923604,0.920743,0.972123,0.978414,0.947234,0.948230,0.002620,0.002097,0.0010
2,3,0.945301,0.940659,0.940340,0.942567,0.972521,0.963972,0.956153,0.952354,0.004294,0.002402,0.0100
3,4,0.949453,0.942857,0.948270,0.945693,0.970529,0.963972,0.959264,0.953849,0.004442,0.002039,0.1000
4,5,0.966303,0.958242,0.969538,0.959502,0.975708,0.974804,0.972607,0.966324,0.004868,0.001880,1.0000
5,6,0.970697,0.962637,0.971238,0.966134,0.981283,0.974961,0.976230,0.969766,0.004924,0.001571,10.0000
6,7,0.971430,0.960440,0.970900,0.959394,0.982876,0.978571,0.976846,0.968258,0.005701,0.001722,100.0000
7,8,0.973629,0.962637,0.972110,0.966134,0.985267,0.974961,0.978639,0.969766,0.006456,0.001859,1000.0000
8,9,0.972407,0.960440,0.970955,0.962609,0.984468,0.974961,0.977659,0.967975,0.007165,0.002273,10000.0000


In [90]:
grid.cv_results_

{'mean_fit_time': array([0.0015106 , 0.00215771, 0.00576615, 0.0022893 , 0.00828891,
        0.00291648, 0.08196244, 0.00509856, 0.20840662, 0.00455713,
        0.45259011, 0.00530467, 0.76346612, 0.00631115, 0.25549161,
        0.00640635, 0.18329368, 0.00654554]),
 'std_fit_time': array([3.75086826e-04, 4.35375794e-04, 1.21965107e-03, 2.92134412e-04,
        5.99114455e-04, 1.15472407e-04, 3.74091317e-02, 6.66956127e-04,
        6.07058691e-02, 4.38461075e-04, 1.33938697e-01, 5.53606228e-04,
        1.90942825e-01, 9.28118406e-04, 1.24469792e-01, 1.17417354e-03,
        5.59974116e-02, 1.41813807e-03]),
 'mean_score_time': array([0.00266047, 0.00152671, 0.00186698, 0.00151944, 0.00169969,
        0.00148959, 0.00264671, 0.00221219, 0.00187354, 0.00170979,
        0.00238853, 0.00159106, 0.00164223, 0.00166147, 0.00227516,
        0.00155947, 0.00230553, 0.00159173]),
 'std_score_time': array([1.15667618e-03, 5.41542231e-05, 6.46147983e-04, 7.87241397e-05,
        3.66567693e-04, 3.95